In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from tensorflow import keras
from PIL import Image
imsize=50
base_model = keras.applications.VGG19(
    weights='imagenet',
    input_shape=(imsize, imsize, 3),
    include_top= False)


## Freeze Base Model

In [ ]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(imsize,imsize, 3))

# Apply the base model to the inputs
x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add additional dense layers with dropout for regularization
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)  # Adding dropout for regularizationa

x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)  # Adding dropout for regularizationa

# x = keras.layers.Dense(64, activation='relu')(x)
# x = keras.layers.Dropout(0.2)(x)  # Adding dropout for regularizationa

# x = keras.layers.Dense(32, activation='relu')(x)
# x = keras.layers.Dropout(0.1)(x)  # Adding dropout for regularization
# Add final dense layer
outputs = keras.layers.Dense(5, activation='softmax')(x)

# Combine inputs and outputs to create the model
model = keras.Model(inputs, outputs)


In [ ]:
model.summary()

## Compile Model

In [ ]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy' , metrics=['accuracy'])

## Augment the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to add random noise to an image 12 2929
def add_noise(img):
    noise = np.random.normal(loc=0, scale=12, size=img.shape)
    img_with_noise = img + noise
    img_with_noise = np.clip(img_with_noise, 0, 255)
    return img_with_noise.astype(np.uint8)

# Function to add pixelation to an image
def add_pixelation(img):
    pixelate = image.img_to_array(image.array_to_img(img).resize((29, 29), resample=image.NEAREST))
    pixelate = image.img_to_array(image.array_to_img(pixelate).resize(img.shape[:-1], resample=image.NEAREST))
    return pixelate.astype(np.uint8)

# Custom ImageDataGenerator with noise and pixelation
class CustomImageDataGenerator(ImageDataGenerator):
    def random_transform(self, x, seed=None):
        x = super().random_transform(x, seed)

        # Add noise
        x = add_noise(x)

        # Add pixelation
        x = add_pixelation(x)

        return x

# Create the generators
datagen_train = CustomImageDataGenerator(
    samplewise_center=True,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)

datagen_valid = ImageDataGenerator(samplewise_center=True)


## Load Dataset

In [ ]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "/content/drive/MyDrive/Rdataset/train/",
    target_size=(imsize, imsize),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "/content/drive/MyDrive/Rdataset/val/",
    target_size=(imsize, imsize),
    color_mode="rgb",
    class_mode="categorical",
)

## Train the Model

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

## Unfreeze Model for Fine Tuning

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

## Evaluate the Model

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

In [ ]:
model.evaluate(train_it, steps=train_it.samples/train_it.batch_size)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(50, 50))
    image = image_utils.img_to_array(image)
    image = image.reshape(1, 50, 50, 3)
    image = preprocess_input(image)
    preds = model.predict(image)

    # Get class labels from the generator
    class_labels = list(train_it.class_indices.keys())

    # Get the predicted class index
    predicted_class_index = tf.argmax(preds, axis=1).numpy()[0]

    # Get the predicted class label
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label

# Assuming your validation images are in the 'dataset/val/' folder
val_folder = 'events'

# Get a list of all image files in the validation folder
image_files = [f for f in os.listdir(val_folder) if f.endswith('.jpeg') or f.endswith('.jpg') or f.endswith('.png')]

# Loop through each image and make predictions
for image_file in image_files:
    image_path = os.path.join(val_folder, image_file)
    predicted_label = make_predictions(image_path)
    print(f"Image: {image_file}, Predicted Label: {predicted_label}")


## Saving Model

In [ ]:
# # save model
# model.save('task4afinal.h5')
# print('Model Saved!')

# model.save_weights('task4afinal')
# print('Weights Saved!')

## Classification Testing

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Load the model architecture and weights
loaded_model = load_model('task4afinal.h5')
loaded_model.load_weights('task4afinal')

def show_image(image):
    if isinstance(image, str):  # Check if image is a file path
        plt.imshow(plt.imread(image))
    elif isinstance(image, np.ndarray):  # Check if image is a NumPy array
        plt.imshow(image)
    else:
        raise ValueError("Unsupported image type")

    plt.show()

def make_predictions_loaded_model(image):
    show_image(image)

    # Convert the image to RGB mode if it has an alpha channel
    if image.shape[-1] == 4:
        image = image[:, :, :3]

    # Resize the image to the expected input size of the model
    image = Image.fromarray(image.astype('uint8'))  # Ensure the image is in uint8 format
    image = image.resize((imsize, imsize))

    image = image_utils.img_to_array(image)
    image = image.reshape(1, imsize, imsize, 3)
    image = preprocess_input(image)
    preds = model.predict(image)

    # Get class labels from the generator
    class_labels = ["combat", "destroyedbuilding", "fire", "humanitarianaid", "militaryvehicles"]

    # Get the predicted class index
    predicted_class_index = tf.argmax(preds, axis=1).numpy()[0]

    # Get the predicted class label
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label

# Example usage with the loaded model
image_path = 'event_4.png'
image = plt.imread(image_path)  # Load the image directly
predicted_label_loaded_model = make_predictions_loaded_model(image)
print("Predicted Label using loaded model:", predicted_label_loaded_model)
